In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras import optimizers

from tensorflow.python.keras.optimizers import TFOptimizer



import numpy as np
np.array(object, dtype=object, copy=True, order='K', subok=False, ndmin=0)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Lambda



import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json' , encoding='utf-8').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

151 documents
42 classes ['Black borders on my monitors screen', 'Blue Sreen issue', 'Booting Up Problem', 'Computer Virus', 'Cookies', 'Crash Problem', 'Customer Support', 'Deleted files', 'E-mail id', 'Email Unable', 'Email attachments', 'Format Virus', 'Hardware Fails', 'Image Distored or Skewed', 'Monitor Fails', 'Mouse', 'Network Problems', 'Network name', 'No signal input', 'Password Reset', 'Printer Installation', 'Printer Jam', 'Printer smudged/distorted pages', 'Printing issues', 'Recover file', 'Safe mode', 'Slow Computer', 'Slow Internet', 'Space in emails', 'Turn off window without doing a shut down', 'USB', 'Unexpected Shut Down', 'Unsaved Document', 'Update', 'Vague', 'Virus', 'Window Defender', 'goodbye', 'greeting', 'noanswer', 'options', 'thanks']
235 unique lemmatized words ["'", "'no", "'s", ',', '.', '@', 'a', 'address', 'admin', 'allowed', 'am', 'an', 'anti', 'any', 'anymore', 'anything', 'are', 'attachment', 'awesome', 'be', 'bin', 'black', 'blue', 'bluetooth', 'b

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word – create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
# create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a ‘0’ for each tag and ‘1’ for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype=object)
# create train and test lists. X – patterns, Y – intents 
#EDITED HERE
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.Adam(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=10, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Train on 151 samples
Epoch 1/300
151/151 [==============================] - 1s 4ms/sample - loss: 3.7484 - accuracy: 0.0265
Epoch 2/300
151/151 [==============================] - 0s 271us/sample - loss: 3.4661 - accuracy: 0.1192
Epoch 3/300
151/151 [==============================] - 0s 264us/sample - loss: 3.0296 - accuracy: 0.2318
Epoch 4/300
151/151 [==============================] - 0s 284us/sample - loss: 2.6586 - accuracy: 0.3046
Epoch 5/300
151/151 [==============================] - 0s 244us/sample - loss: 2.1170 - accuracy: 0.4768
Epoch 6/300
151/151 [==============================] - 0s 244us/sample - loss: 1.6610 - accuracy: 0.5828
Epoch 7/300
151/151 [==============================] - 0s 231us/sample - loss: 1.4673 - accuracy: 0.5629
Epoch 8/300
151/151 [==============================] - 0s 271us/sample - loss: 1.2131 - accuracy: 0.6358
Epoch 9/300
151/151 [==============================] - 0s 317us/sample - loss: 0.9407 - accuracy: 0.7682
Epoch 10/300
151/151 [==============

151/151 [==============================] - 0s 244us/sample - loss: 0.4264 - accuracy: 0.8808
Epoch 78/300
151/151 [==============================] - 0s 231us/sample - loss: 0.6217 - accuracy: 0.8874
Epoch 79/300
151/151 [==============================] - 0s 238us/sample - loss: 0.6898 - accuracy: 0.8742
Epoch 80/300
151/151 [==============================] - 0s 244us/sample - loss: 0.6451 - accuracy: 0.8609
Epoch 81/300
151/151 [==============================] - 0s 225us/sample - loss: 0.5145 - accuracy: 0.8543
Epoch 82/300
151/151 [==============================] - 0s 238us/sample - loss: 0.6017 - accuracy: 0.8609
Epoch 83/300
151/151 [==============================] - 0s 231us/sample - loss: 0.9028 - accuracy: 0.8344
Epoch 84/300
151/151 [==============================] - 0s 225us/sample - loss: 0.5987 - accuracy: 0.8808
Epoch 85/300
151/151 [==============================] - 0s 244us/sample - loss: 0.3944 - accuracy: 0.8742
Epoch 86/300
151/151 [==============================] - 0s 

151/151 [==============================] - 0s 222us/sample - loss: 0.2621 - accuracy: 0.9007
Epoch 154/300
151/151 [==============================] - 0s 218us/sample - loss: 0.6089 - accuracy: 0.9007
Epoch 155/300
151/151 [==============================] - 0s 225us/sample - loss: 0.4193 - accuracy: 0.9139
Epoch 156/300
151/151 [==============================] - 0s 248us/sample - loss: 0.5224 - accuracy: 0.9073
Epoch 157/300
151/151 [==============================] - 0s 218us/sample - loss: 0.3944 - accuracy: 0.8742
Epoch 158/300
151/151 [==============================] - 0s 225us/sample - loss: 0.6964 - accuracy: 0.8742
Epoch 159/300
151/151 [==============================] - 0s 211us/sample - loss: 0.6018 - accuracy: 0.8675
Epoch 160/300
151/151 [==============================] - 0s 211us/sample - loss: 0.8743 - accuracy: 0.8543
Epoch 161/300
151/151 [==============================] - 0s 205us/sample - loss: 0.8757 - accuracy: 0.8278
Epoch 162/300
151/151 [============================

151/151 [==============================] - 0s 218us/sample - loss: 0.3657 - accuracy: 0.9007
Epoch 230/300
151/151 [==============================] - 0s 218us/sample - loss: 0.4513 - accuracy: 0.8940
Epoch 231/300
151/151 [==============================] - 0s 225us/sample - loss: 0.3915 - accuracy: 0.8940
Epoch 232/300
151/151 [==============================] - 0s 218us/sample - loss: 0.4801 - accuracy: 0.8808
Epoch 233/300
151/151 [==============================] - 0s 218us/sample - loss: 0.2092 - accuracy: 0.9470
Epoch 234/300
151/151 [==============================] - 0s 238us/sample - loss: 0.7022 - accuracy: 0.8808
Epoch 235/300
151/151 [==============================] - 0s 218us/sample - loss: 0.4046 - accuracy: 0.9272
Epoch 236/300
151/151 [==============================] - 0s 231us/sample - loss: 0.5136 - accuracy: 0.9007
Epoch 237/300
151/151 [==============================] - 0s 211us/sample - loss: 0.4831 - accuracy: 0.8874
Epoch 238/300
151/151 [============================

SyntaxError: invalid syntax (<ipython-input-8-93f230217d77>, line 1)